In [ ]:
import uproot as up
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
import re
import pandas as pd
from uproot3_methods.classes.TLorentzVector import TLorentzVector
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
file = up.open('../build/evtOutput.root')
ntp = file["ntp"]

In [ ]:
df = pd.DataFrame(index = ak.to_pandas(ntp["pz"].array()).index, columns=["px", "py", "pz", "E"])
df["px"] = ak.to_pandas(ntp["px"].array())
df["py"] = ak.to_pandas(ntp["py"].array())
df["pz"] = ak.to_pandas(ntp["pz"].array())
df["E"] = ak.to_pandas(ntp["E"].array())
df = df.reset_index()

In [ ]:
df_pivoted = pd.pivot_table(df, index = "entry", columns="subentry", values = ["px","py", "pz","E"])
# df_pivoted.columns = [c[0]+"_"+str(c[1]) for c in df_pivoted.columns]
df_pivoted

In [ ]:
def parse_var(var):
        [name, numbers] = var.split("_")
        func = {'m2': lambda t, x, y, z: t**2-x**2-y**2-z**2,
                'm': lambda t, x, y, z: np.sqrt(t**2-x**2-y**2-z**2),
                'pt2': lambda t, x, y, z: x**2+y**2,
                 'pt': lambda t, x, y, z: np.sqrt(x**2+y**2)}[name]
        part_list = [int(n) for n in re.sub('([0-9])',r'\1 ',numbers.replace('m','-')).split(' ')[:-1]]
        return func, part_list

In [ ]:
def sign(x):
    return -1 if x<0 else 1

In [ ]:
df_res = pd.DataFrame(index=df_pivoted.index, columns = ["px","py","pz","E", "out"]).fillna(0)
var = "m_123"
func, part_list = parse_var(var)
for i in part_list:
    abs_i = abs(i)
    for v in ["px","py","pz","E"]:
        df_res[v] += sign(i)*df_pivoted[v,abs_i]
df_res["out"] = func( df_res["E"], df_res["px"], df_res["py"], df_res["pz"])